In [1]:
import numpy as np
import pandas as pd
import os
import sys
import pickle
import ipywidgets as widgets

import plotly.express as px

from IPython.display import clear_output

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from custom_scripts.config import *
from custom_scripts.prepare_data import prepare_data

from sklearn.manifold import TSNE

from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.covariance import EllipticEnvelope

<h2 align="middle"><b><font size="10">ANOMALY DETECTION</b></h2> 

In [19]:

label = widgets.HTML(
    value="<font size='3'> Given a network traffic activity, this page will show if there is any cyber attacks on the network. It can detect the following type of cyber attacks <ul> <li>Brute Force</li> <li>Denial of Service (DoS)</li> <li>Distributed Denial of Service (DDoS)</li> </ul>",
    align= 'center',
    positioning = 'top',
    placeholder='',
    description='',
)

display(label)

HTML(value="<font size='3'> Given a network traffic activity, this page will show if there is any cyber attack…

In [20]:

label = widgets.HTML(
    value="<font size='3'>If an anomaly detected in the network, then it is represented by <b><font color='limegreen'>Green</b></font> dot and if no anomaly is detected, then it is represented by <b><font color='blue'>Blue</b></font> dot ",
    align= 'center',
    positioning = 'top',
    placeholder='',
    description='',
)

display(label)

HTML(value="<font size='3'>If an anomaly detected in the network, then it is represented by <b><font color='li…

<h2 align="left">Which day of the week do you want to analyze?</h2> 

In [29]:
selection = widgets.RadioButtons(
    options=['Tuesday','Wednesday'],
    description='Day of Week:',
    disabled=False
)
display(selection)

RadioButtons(description='Day of Week:', layout=Layout(height='100px', width='80%'), options=('Tuesday', 'Wedn…

<h2 align="left">What fraction of total network traffic in the selected day do you want to inspect?</h2> 



<h5 align="left">Note: All network traffic might take some time to load. Suggested value is 0.1</h5> 

In [5]:
sample_size =widgets.FloatSlider(
    value=0.1,
    min=0,
    max=1,
    step=0.01,
    description='Test:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

display(sample_size)

FloatSlider(value=0.1, continuous_update=False, description='Test:', max=1.0, step=0.01)

In [10]:
go_button =widgets.Button(
    description='DETECT',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to perform anomaly detection',
    icon=''
)
go_output = widgets.Output()

def go_button_clicked(b):
    with go_output:
        clear_output()
        anomaly_detector(selection,sample_size)
        
go_button.on_click(go_button_clicked)

display(go_button,go_output)

Button(button_style='success', description='DETECT', style=ButtonStyle(), tooltip='Click to perform anomaly de…

Output()

In [7]:
def anomaly_detector(selection, sample_size, method='Mixed', predict='False',model='lof'):
    
    day_of_week = selection.value
    frac = sample_size.value    
    
    try:
    
        print('Loading predictions...')
        filename='../notebooks/predictions/novelty/'+day_of_week+'/'+model+'_anomalies_'+method+'.pkl'
    
        with open(filename,'rb') as file:
            anomalies=pickle.load(file)
        
        print('Predictions loaded!')
    
    except:
        
        _, tue_test_data, wed_test_data = novelty_detection_preparation(method=method)
    
        X_tue_test, _ = prepare_data(data=tue_test_data,class_column='Label',classes='binary',neg_class='BENIGN')
        X_wed_test, _ = prepare_data(data=wed_test_data,class_column='Label',classes='binary',neg_class='BENIGN')
    
        X_test ={'Tuesday':X_tue_test,'Wednesday':X_wed_test}      
    
        filename = '../notebooks/models/novelty/'+model+'_model_'+method+'.pkl'
    
        with open(filename,'rb') as file:
            anomaly_model=pickle.load(file)
    
        print('Making predictions...')
    
        anomalies = anomaly_model.predict(X_test[day_of_week])
    
        filename_2 = '../notebooks/predictions/novelty/'+day_of_week+'/'+model+'_anomalies_'+method+'.pkl'
    
        with open(filename_2,'wb') as file:
            pickle.dump(anomalies,file)   
        
        print('Predictions made! Also saved for future reference.')
        
    anomalies = (anomalies*-1 +1)*0.5
    anomalies = anomalies.astype(int)
      
    vis_filename='../notebooks/tsne_transforms/novelty/'+day_of_week+'_perplexity_30_anomaly.pkl'
    with open(vis_filename,'rb') as file:
        x_vis=pickle.load(file)
        
    visual_sample = int(x_vis.shape[0]*frac)
    index = np.random.randint(0,x_vis.shape[0],visual_sample)
    x_red_vis = x_vis[index,:]    
    
    x_plotly = pd.DataFrame(x_red_vis,columns=['dim1','dim2'])
    x_plotly['anomaly']=anomalies[index]
    anomaly_dict={1:'Anomaly',0:'Normal'}
    x_plotly['Label']=x_plotly['anomaly'].map(anomaly_dict)
        
    fig = px.scatter(x_plotly, 
                     x = "dim1",
                     y = 'dim2',
                     color = 'Label',
                     color_discrete_map = {'Anomaly':'limegreen','Normal':'blue'},
                     hover_name = 'Label',
                     labels = {'dim1':'Latent Dimension 1','dim2':'Latent Dimension 2'},
                     title = 'Anomaly Detection Results for '+day_of_week+' Network Traffic',
                     render_mode = 'webgl',
                     width = 900,
                     height = 600,
                     template = "plotly_dark"
                    )
    
    fig.update(layout={'title': {'x':0.5,
                             'font':{'size':24}
                            }, 
                   'legend':{
                             'font':{'size':18}
                            },
                   'xaxis':{'title':
                            {
                             'font':{'size':18}
                            }
                            },
                   'yaxis':{'title':
                            {
                             'font':{'size':18}
                            }
                            }
                      }
              )
    
    
    fig.show()